In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import jieba
import re
from collections import Counter
import csv

from tqdm import tqdm



from tqdm import tqdm_notebook
from scipy.linalg import norm

from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
from gensim.models.word2vec import LineSentence




import jieba  
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer  

import pyecharts.options as opts
from pyecharts.charts import WordCloud

In [2]:
import warnings
warnings.filterwarnings('ignore')

# 一、各景区词云表

In [3]:
content = pd.read_excel(r"./附件1/景区评论.xlsx",engine='openpyxl')  
content.head()

,景区名称,评论日期,评论内容
0,A01,2020-06-16,是亲子游的绝佳场所，门票就是有点贵，不过可以接受，爷爷奶奶不放心小朋友也跟上来了，当天我们十...
1,A01,2020-01-23,**景区差不多，票价偏贵了。大马戏比较精彩，八点的场次，6点40才能检票进入，我们6点多看看...
2,A01,2020-03-22,很有**特色的亲子酒店，房间里的装修很可爱，小孩子特别喜欢，洗漱用品也很有特色，对应的房间还...
3,A01,2020-12-25,有园区的工作人员在那，他会主动给你园区里的地图和表演的时间安排，很周到，上接驳车大概也是34...
4,A01,2020-11-28,周五逃课跟朋友在广州集合！终于如愿以偿的到达欢乐世界。学生票198 需要出示相关证件（校卡或...


In [24]:
def filter_punc(sentence):
    sentence = re.sub("[\s+\.\!\/_,$%^*(+\"\'“”《》?“]+|[+——！，。？、~@#￥%……&*（）：]+", "", sentence)
    return sentence

In [25]:

#使用停用词
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords


In [26]:
def scenic_spot_participle(scenic_spot_name):
    stopwords = stopwordslist('hit_stopwords.txt')
    words = jieba.lcut(' '.join(content.loc[content['景区名称']==scenic_spot_name]['评论内容'].values))
    for i in tqdm(words):
            if i in stopwords:
                words.remove(i)
    words = ' '.join(words)
    return words

In [27]:
def words_name(words_name):
    vectorizer=CountVectorizer()         #该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频  
    transformer=TfidfTransformer()       #该类会统计每个词语的tf-idf权值  
    X=vectorizer.fit_transform([words_name])   #将文本转为词频矩阵
    tfidf=transformer.fit_transform(X)   #计算tf-idf，  
    word=vectorizer.get_feature_names()  #获取词袋模型中的所有词语   
    weight=tfidf.toarray()               #将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重  

    spec = []
    for  w in  weight:
            loc = np.argsort(-w)
            for i in range(20):
                print(u'-{}: {} {}'.format(str(i + 1), word[loc[i]], w[loc[i]]))
                spec.append((word[loc[i]], str(w[loc[i]])))
    return spec

In [7]:
scenic_name = list(content.groupby('景区名称'))
names = []
for name in scenic_name:
    name = name[0]
    names.append(name)
# names

In [9]:
for name in names:
    words = scenic_spot_participle(name)
    words_spec = words_name(words)
    with open(r'./热门词'+'/'+'景区'+'/'+name+'.csv','w',encoding='utf-8',newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['评论热词','热度'])

        writer.writerows(words_spec)
    print(name+'保存成功')

 71%|███████████████████████████████████████████████████▍                    | 134367/187931 [00:59<00:23, 2261.95it/s]


-1: 方便 0.3270192600460427
-2: 取票 0.26623161282574404
-3: 动物 0.2545768383586943
-4: 非常 0.23012466447684485
-5: 不错 0.21641316510384515
-6: 表演 0.18396261658774588
-7: 很多 0.17962064178629597
-8: 好玩 0.16636619239239628
-9: 马戏 0.15516846790444652
-10: 里面 0.14602746832244673
-11: 开心 0.14122844354189684
-12: 值得 0.12774546915844714
-13: 动物园 0.12728841917934716
-14: 景区 0.12568874425249718
-15: 项目 0.11791889460779736
-16: 排队 0.11700479464959739
-17: 孩子 0.11654774467049739
-18: 过山车 0.11517659473319743
-19: 真的 0.11266281984814748
-20: 刺激 0.11152019490039751
A01保存成功


 72%|███████████████████████████████████████████████████▋                    | 115168/160535 [01:12<00:28, 1597.13it/s]


-1: 表演 0.31850846069786865
-2: 不错 0.24688097842346385
-3: 非常 0.2367891338418467
-4: 方便 0.21955915528786615
-5: 取票 0.19174504705072618
-6: 好玩 0.18657605348453202
-7: 值得 0.1850991981799051
-8: 海洋 0.17156135788749183
-9: 很多 0.15826966014584973
-10: 里面 0.15654666229045167
-11: 烟花 0.1558082346381382
-12: 晚上 0.15359295168119785
-13: 孩子 0.14079353904109804
-14: 开心 0.1395628262872423
-15: 排队 0.13808597098261538
-16: 动物 0.13144012211179432
-17: 项目 0.12430198813943096
-18: 一天 0.12036370732709255
-19: 时间 0.116179283963983
-20: 过山车 0.10805657978853503
A02保存成功


 72%|████████████████████████████████████████████████████▊                    | 81349/112493 [00:33<00:12, 2429.97it/s]


...

-1: 不错 0.47746472458120504
-2: 五台山 0.33096986590288074
-3: 取票 0.3092669238764623
-4: 景区 0.28756398185004395
-5: 方便 0.2658610398236255
-6: 导游 0.2007522137443703
-7: 地方 0.15192059418492887
-8: 佛教 0.14106912317171966
-9: 非常 0.13021765215851044
-10: 门票 0.12479191665190585
-11: 高速 0.09766323911888285
-12: 时间 0.09223750361227824
-13: 五爷 0.09223750361227824
-14: 圣地 0.08681176810567363
-15: 服务 0.08138603259906904
-16: 值得 0.08138603259906904
-17: 比较 0.08138603259906904
-18: 一定 0.07053456158585983
-19: 没有 0.07053456158585983
-20: 讲解 0.06510882607925522
A48保存成功


 41%|██████████████████████████████▋                                            | 3813/9321 [00:00<00:00, 37859.07it/s]

-1: 不错 0.32934197342301713
-2: 观音 0.27944167441952966
-3: 景区 0.254491524917786
-4: 莲花 0.2495014950174372
-5: 方便 0.17465104651220606
-6: 值得 0.1696610166118573
-7: 很多 0.16467098671150857
-8: 风景 0.16467098671150857
-9: 非常 0.14970089701046232
-10: 采石场 0.14471086711011358
-11: 广州 0.13473080730941608
-12: 景色 0.12974077740906736
-13: 门票 0.1247507475087186
-14: 地方 0.1247507475087186
-15: 比较 0.11976071760836987
-16: 望海 0.11477068770802112
-17: 景点 0.10978065780767238
-18: 里面 0.10978065780767238
-19: 特别 0.10479062790732363
-20: 环境 0.09980059800697488
A49保存成功


 71%|█████████████████████████████████████████████████████▍                     | 6648/9321 [00:00<00:00, 29590.83it/s]

-1: 景区 0.3344210243482231
-2: 瀑布群 0.2554605047104482
-3: 黄山 0.24152629536260556
-4: 不错 0.20901314021763942
-5: 公园 0.1718552486233924
-6: 值得 0.1718552486233924
-7: 非常 0.16256577572483066
-8: 空气 0.16256577572483066
-9: 景点 0.15327630282626892
-10: 水谷 0.14863156637698804
-11: 方便 0.14398682992770717
-12: 景色 0.14398682992770717
-13: 地方 0.14398682992770717
-14: 瀑布 0.13005262057986453
-15: 广东 0.13005262057986453
-16: 门票 0.13005262057986453
-17: 适合 0.13005262057986453
-18: 风景 0.12540788413058365
-19: 我们 0.12540788413058365
-20: 酒店 0.11147367478274103
A50保存成功


# 一、各酒店词云表

In [10]:
content_hotel = pd.read_excel(r"./附件1/酒店评论.xlsx",engine='openpyxl')  
content_hotel.head()

,酒店名称,评论日期,评论内容,入住房型
0,H01,2020-01-01,酒店很适合家庭出行,标准客房
1,H01,2020-01-01,升级了房间 延迟退房 很赞,标准客房
2,H01,2020-01-01,这几年，每年都会来广州，每次都会住**酒店。因为位置好，酒店的性价比也不错，这次给免费升级了...,标准客房
3,H01,2020-01-01,酒店很好不错,标准客房
4,H01,2020-01-01,超五星好评,高级客房


In [28]:
def scenic_spot_participle(scenic_spot_name):
    stopwords = stopwordslist('hit_stopwords.txt')
    line = filter_punc(' '.join(content_hotel.loc[content_hotel['酒店名称']==scenic_spot_name]['评论内容'].values))
    words = jieba.lcut(line)
   
    for i in tqdm(words):
            if i in stopwords:
                words.remove(i)
    words = ' '.join(words)
    return words

In [29]:
def words_name(words_name):
    vectorizer=CountVectorizer()         #该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频  
    transformer=TfidfTransformer()       #该类会统计每个词语的tf-idf权值  
    X=vectorizer.fit_transform([words_name])   #将文本转为词频矩阵
    tfidf=transformer.fit_transform(X)   #计算tf-idf，  
    word=vectorizer.get_feature_names()  #获取词袋模型中的所有词语   
    weight=tfidf.toarray()               #将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重  

    spec = []
    for  w in  weight:
            loc = np.argsort(-w)
            for i in range(20):
                print(u'-{}: {} {}'.format(str(i + 1), word[loc[i]], w[loc[i]]))
                spec.append((word[loc[i]], str(w[loc[i]])))
    return spec

In [30]:
scenic_hotel_name = list(content_hotel.groupby('酒店名称'))
hotel_names = []
for name in scenic_hotel_name:
    name = name[0]
    hotel_names.append(name)

In [31]:
for name in hotel_names:
    words = scenic_spot_participle(name)
    words_spec = words_name(words)
    with open(r'./热门词'+'/'+'酒店'+'/'+name+'.csv','w',encoding='gbk',newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['评论热词','热度'])

        writer.writerows(words_spec)
    print(name+'保存成功')

  0%|                                                                                        | 0/14312 [00:00<?, ?it/s]

-1: 不错 0.5274653948396629
-2: 酒店 0.47484414404805764
-3: 服务 0.365842981694018
-4: 方便 0.31196884397880303
-5: 位置 0.19043881238866692
-6: 非常 0.18417437777061865
-7: 房间 0.15911663929842565
-8: 交通 0.14658777006232915
-9: 前台 0.12278291851374577
-10: 环境 0.11526559697208787
-11: 干净 0.10148384081238171
-12: 早餐 0.0939665192707238
-13: 便利 0.0914607454235045
-14: 性价比 0.0814376500346273
-15: 东站 0.08018476311101765
-16: 卫生 0.0714145546457501
-17: 服务态度 0.07016166772214044
-18: 入住 0.07016166772214044
-19: 地理位置 0.06389723310409219
-20: 设施 0.05262125079160533
H01保存成功


  0%|                                                                                        | 0/16620 [00:00<?, ?it/s]

-1: 酒店 0.5791121458489736
-2: 服务 0.5404349808735006
-3: 不错 0.31046264858690464
-4: 非常 0.2101110854072991
-5: 前台 0.19338582487736483
-6: 房间 0.14843668720316652
-7: 方便 0.14530070085380384
-8: 入住 0.12230346762514425
-9: 早餐 0.12125813884202336
-10: 位置 0.11185017979393534
-11: 环境 0.07839965873406683
-12: 深圳 0.07735432995094593
-13: 干净 0.07317301481846238
-14: 交通 0.07003702846909969
-15: 特别 0.06376505577037435
-16: 下次 0.06062906942101168
-17: 热情 0.05540242550540722
-18: 比较 0.05226643915604455
-19: 卫生 0.05122111037292366
-20: 性价比 0.048085124023560986
H02保存成功


  0%|                                                                                        | 0/19558 [00:00<?, ?it/s]

...

-1: 酒店 0.6112622061166647
-2: 不错 0.2813195380423286
-3: 房间 0.25700797302632494
-4: 方便 0.25700797302632494
-5: 服务 0.22575024657717732
-6: 非常 0.21533100442746142
-7: 位置 0.15281555152916618
-8: 老牌 0.1354501479463064
-9: 设施 0.1180847443634466
-10: 公园 0.1180847443634466
-11: 入住 0.1180847443634466
-12: 交通 0.11461166364687464
-13: 环境 0.10419242149715875
-14: 五星 0.10419242149715875
-15: 越秀 0.1007193407805868
-16: 早餐 0.09724626006401484
-17: 广州 0.09377317934744288
-18: 性价比 0.08682701791429896
-19: 感觉 0.07988085648115505
-20: 比较 0.07988085648115505
H47保存成功
-1: 服务 0.5584669808598092
-2: 酒店 0.3862295007815503
-3: 不错 0.3288170074221306
-4: 卫生 0.31315905468774347
-5: 房间 0.25052724375019475
-6: 位置 0.19833406796890418
-7: 方便 0.19311475039077514
-8: 非常 0.15136020976574266
-9: 干净 0.1357022570313555
-10: 环境 0.10960566914071021
-11: 前台 0.10438635156258115
-12: 地理位置 0.09916703398445209
-13: 交通 0.09394771640632303
-14: 服务态度 0.09394771640632303
-15: 性价比 0.08872839882819397
-16: 便利 0.08350908125006493
-17: 下次

  0%|                                                                                         | 0/4511 [00:00<?, ?it/s]

-1: 酒店 0.5179038956593259
-2: 服务 0.32813758274598515
-3: 不错 0.30046332877945625
-4: 非常 0.26883560996056616
-5: 前台 0.237207891141676
-6: 早餐 0.23325442628931473
-7: 房间 0.22139403173223093
-8: 方便 0.1897663129133408
-9: 位置 0.16209205894681195
-10: 入住 0.15023166438972813
-11: 特别 0.11069701586611547
-12: 体验 0.09883662130903166
-13: 感觉 0.09883662130903166
-14: 干净 0.08697622675194787
-15: 卫生 0.08697622675194787
-16: 热情 0.0830227618995866
-17: 比较 0.0830227618995866
-18: 很棒 0.0830227618995866
-19: 环境 0.07906929704722533
-20: 交通 0.0711623673425028
H49保存成功


 81%|████████████████████████████████████████████████████████████▊              | 3661/4511 [00:00<00:00, 33676.74it/s]

-1: 酒店 0.6231025240803547
-2: 不错 0.3441554638815913
-3: 方便 0.2608336147313113
-4: 房间 0.23547479107687824
-5: 服务 0.19200252195499304
-6: 非常 0.19200252195499304
-7: 位置 0.17388907648754084
-8: 比较 0.16664369830056
-9: 早餐 0.16302100920706955
-10: 设施 0.14128487464612693
-11: 交通 0.12679411827216522
-12: 干净 0.11954874008518435
-13: 汕头 0.1050579837112226
-14: 机场 0.10143529461773217
-15: 环境 0.09056722733726086
-16: 卫生 0.07969916005678956
-17: 老牌 0.07607647096329913
-18: 没有 0.06883109277631826
-19: 大巴 0.06883109277631826
-20: 性价比 0.06883109277631826
H50保存成功
